In [3]:
%matplotlib inline
# -*- coding: utf-8 -*-
"""
Exemple de MNIST avec PyTorch
Exemple avec AlexNet
Version avec GPU
"""
import torch
torch.manual_seed(0) # Pour résultats reproductibles

# Fonction J d'entropie croisée
import torch.nn.functional as F
fonction_cout = F.cross_entropy

def taux_bonnes_predictions(lot_Y_predictions, lot_Y):
    predictions_categorie = torch.argmax(lot_Y_predictions, dim=1)
    return (predictions_categorie == lot_Y).float().mean()

from torch import nn
# Définition de l'architecture du RNA

class BlocResiduel2d(nn.Module):
    """ Représente un bloc de base de l'architecture Resnet
    nb_canaux_in : nombre de canaux de l'entrée X
    nb_canaux_out : nombre de canaux de la sortie Y
    pas : le pas de la première convolution conv1
    """
    def __init__(self, nb_canaux_in, nb_canaux_out,pas=1):
        super().__init__()
        self.conv1 = nn.Conv2d(nb_canaux_in, nb_canaux_out,kernel_size=3, padding=1, stride=pas)
        self.conv2 = nn.Conv2d(nb_canaux_out, nb_canaux_out,kernel_size=3, padding=1)
        if nb_canaux_in != nb_canaux_out:
            self.conv3 = nn.Conv2d(nb_canaux_in, nb_canaux_out,kernel_size=1, stride=pas)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(nb_canaux_out)
        self.bn2 = nn.BatchNorm2d(nb_canaux_out)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, lot_X):
        lot_Y_predictions = F.relu(self.bn1(self.conv1(lot_X)))
        lot_Y_predictions = self.bn2(self.conv2(lot_Y_predictions))
        
        # si le nombre de canaux de la sortie est différent de l'entrée
        # la convolution 1x1 fait la conversion de l'entrée X
        if self.conv3:
            lot_X = self.conv3(lot_X)

        lot_Y_predictions += lot_X
        return F.relu(lot_Y_predictions)
    
# Resnet avec 18 couches convolutives
modele1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), # ->(N,64,112,112)
                   nn.BatchNorm2d(64), nn.ReLU(), # ->(N,64,112,112)
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1),# ->(N,64,56,56)
                   BlocResiduel2d(64,64), BlocResiduel2d(64,64) ,# ->(N,64,56,56)
                   BlocResiduel2d(64,128,2), BlocResiduel2d(128,128),# ->(N,128,28,28)
                   BlocResiduel2d(128,256,2), BlocResiduel2d(256,256),# ->(N,256,14,14)
                   BlocResiduel2d(256,512,2), BlocResiduel2d(512,512),# ->(N,512,7,7)
                   nn.AdaptiveMaxPool2d((1,1)), nn.Flatten(), nn.Linear(512,10)
                  )

modele = nn.Sequential(
    # Here, we use a larger 11 x 11 window to capture objects. At the same
    # time, we use a stride of 4 to greatly reduce the height and width of the
    # output. Here, the number of output channels is much larger than that in
    # LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # Make the convolution window smaller, set padding to 2 for consistent
    # height and width across the input and output, and increase the number of
    # output channels
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # Use three successive convolutional layers and a smaller convolution
    # window. Except for the final convolutional layer, the number of output
    # channels is further increased. Pooling layers are not used to reduce the
    # height and width of input after the first two convolutional layers
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # Here, the number of outputs of the fully-connected layer is several
    # times larger than that in LeNet. Use the dropout layer to mitigate
    # overfitting
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # Output layer. Since we are using Fashion-MNIST, the number of classes is
    # 10, instead of 1000 as in the paper
    nn.Linear(4096, 10))


from torch import optim
optimiseur = optim.SGD(modele.parameters(), lr=0.01)

import torchvision
from torchvision import datasets, models, transforms

pretraitement = transforms.Compose([
transforms.Resize(224),
transforms.ToTensor()])

#Chargement des données
ds = torchvision.datasets.MNIST(root = "./data", train = True, download = True, transform = pretraitement)
ds_ent, ds_valid = torch.utils.data.random_split(ds, [50000, 10000])
ds_test = torchvision.datasets.MNIST(root = "./data", train = False, download = True, transform = pretraitement)

#Création du DataLoader avec le dataset
dl_ent = torch.utils.data.DataLoader(ds_ent, batch_size=100, shuffle = True)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=100)

# Déterminer si un GPU est disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Placer le modèle en mode GPU si possible
modele = modele.to(device)
print('Entrainement sur ',device)

import time
def entrainer_GPU(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10):

    debut = time.time()

    # Listes pour les métriques par epoch
    liste_cout_moyen_ent = []
    liste_taux_moyen_ent = []
    liste_cout_moyen_valid = []
    liste_taux_moyen_valid = []
    
    # Boucle d'apprentissage
    for epoch in range(nb_epochs):
        cout_total_ent = 0 # pour cumuler les couts par mini-lot
        taux_bonnes_predictions_ent = 0 # pour cumuler les taux par mini-lot
        modele.train() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        
        # Boucle d'apprentissage par mini-lot pour une epoch
        for lot_X, lot_Y in dl_ent:
            
            # Les données doivent être placés en GPU pour être compatibles avec le modèle
            lot_X = lot_X.to(device)
            lot_Y = lot_Y.to(device)

            optimiseur.zero_grad() # Remettre les dérivées à zéro
            lot_Y_predictions = modele(lot_X) # Appel de la méthode forward
            cout = fonction_cout(lot_Y_predictions, lot_Y)
            cout.backward() # Calcul des gradiants par rétropropagation
            with torch.no_grad():
                cout_total_ent +=cout
                taux_bonnes_predictions_ent += taux_bonnes_predictions(lot_Y_predictions, lot_Y)
            optimiseur.step() # Mise à jour des paramètres
            # scheduler.step()
        # Calculer les moyennes par mini-lot
        with torch.no_grad():
            cout_moyen_ent = cout_total_ent/len(dl_ent)
            taux_moyen_ent = taux_bonnes_predictions_ent/len(dl_ent)
       
        modele.eval() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        with torch.no_grad():
            # Les données de validation doivent aussi être placés en GPU
            cout_valid = sum(fonction_cout(modele(lot_valid_X.to(device)), lot_valid_Y.to(device)) for lot_valid_X, lot_valid_Y in dl_valid)
            taux_bons_valid = sum(taux_bonnes_predictions(modele(lot_valid_X.to(device)), lot_valid_Y.to(device)) for lot_valid_X, lot_valid_Y in dl_valid)
        cout_moyen_valid = cout_valid/len(dl_valid)
        taux_moyen_valid = taux_bons_valid/len(dl_valid)
        print(f'-------- > epoch {epoch+1}:  coût moyen entraînement = {cout_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  taux moyen entraînement = {taux_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  coût moyen validation = {cout_moyen_valid}')
        print(f'-------- > epoch {epoch+1}:  taux moyen validation = {taux_moyen_valid}')
    
        liste_cout_moyen_ent.append(cout_moyen_ent)
        liste_taux_moyen_ent.append(taux_moyen_ent)
        liste_cout_moyen_valid.append(cout_moyen_valid)
        liste_taux_moyen_valid.append(taux_moyen_valid)
        
        temps_ecoule = time.time() - debut
        print('Temps écoulé : {:.0f}m {:.0f}s'.format(temps_ecoule // 60, temps_ecoule % 60))

    
    # Affichage du graphique d'évolution des métriques par epoch
    import numpy as np
    import matplotlib.pyplot as plt
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_ent,label='Erreur entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_valid,label='Erreur validation')
    plt.title("Evolution du coût")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='upper center')
    plt.show()
        
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_ent,label='Taux bonnes réponses entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_valid,label='Taux bonnes réponses validation')
    plt.title("Evolution du taux")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='center')
    plt.show()

entrainer_GPU(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10)

Entrainement sur  cpu


KeyboardInterrupt: 

In [4]:
X = torch.randn(size=(1, 1, 224, 224), dtype=torch.float32)
for une_couche in modele:
    X = une_couche(X)
    print('Type de couche:',une_couche.__class__.__name__,'  Forme de sortie:',X.shape)


Type de couche: Conv2d   Forme de sortie: torch.Size([1, 96, 54, 54])
Type de couche: ReLU   Forme de sortie: torch.Size([1, 96, 54, 54])
Type de couche: MaxPool2d   Forme de sortie: torch.Size([1, 96, 26, 26])
Type de couche: Conv2d   Forme de sortie: torch.Size([1, 256, 26, 26])
Type de couche: ReLU   Forme de sortie: torch.Size([1, 256, 26, 26])
Type de couche: MaxPool2d   Forme de sortie: torch.Size([1, 256, 12, 12])
Type de couche: Conv2d   Forme de sortie: torch.Size([1, 384, 12, 12])
Type de couche: ReLU   Forme de sortie: torch.Size([1, 384, 12, 12])
Type de couche: Conv2d   Forme de sortie: torch.Size([1, 384, 12, 12])
Type de couche: ReLU   Forme de sortie: torch.Size([1, 384, 12, 12])
Type de couche: Conv2d   Forme de sortie: torch.Size([1, 256, 12, 12])
Type de couche: ReLU   Forme de sortie: torch.Size([1, 256, 12, 12])
Type de couche: MaxPool2d   Forme de sortie: torch.Size([1, 256, 5, 5])
Type de couche: Flatten   Forme de sortie: torch.Size([1, 6400])
Type de couche: Li